In [ ]:
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
from ast import literal_eval

In [ ]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

In [ ]:
CF_clean = pd.read_csv('CareersFutureCleaned.csv')

In [ ]:
CF_clean

In [ ]:
CF_clean.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
CF_clean

In [ ]:
CF_clean['Industry'].unique()

In [ ]:
cf_ind = CF_clean['Industry'].unique()

In [ ]:
cf_ind

In [ ]:
SFW_16  = pd.read_csv('Enhanced construct - 16 sectors.xlsx - Job Role_TSC.csv', error_bad_lines=False)

In [ ]:
SFW_16

In [ ]:
SFW_16['Sector'].unique()

In [ ]:
sfw_ind = SFW_16['Sector'].unique()

In [ ]:
sfw_ind

In [ ]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag, wordnet

# Spacy for industry mapping

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
doc1 = nlp("Design")
doc2 = nlp("Design")
doc1.similarity(doc2)

In [ ]:
' '.join(eval(cf_ind[0])).lower()

In [ ]:
sim = {'cf_ind': [], 'sfw_ind': [], 'sim score': []}
# scores = []
# sfw = []
for i in cf_ind:
    scores = []
    sfw = []
    for j in sfw_ind:
        i_lower = nlp(' '.join(eval(i)).lower())
        j_lower = nlp(j.lower())
        scores.append(i_lower.similarity(j_lower))
        sfw.append(j_lower)
    sim['sim score'].append(scores)
    sim['cf_ind'].append(i_lower)
    sim['sfw_ind'].append(sfw)
    

In [ ]:
sim_df = pd.DataFrame(sim)

In [ ]:
sim_df

In [ ]:
max_scores = []
max_sfw = []
for i in range(0, len(sim_df)):
    max_scores.append(max(sim_df['sim score'][i]))
    sf = np.argmax(sim_df['sim score'][i])
    max_sfw.append(sim_df['sfw_ind'][i][sf])

In [ ]:
len(max_scores)

In [ ]:
len(max_sfw)

In [ ]:
sim_df['max scores'] = max_scores

In [ ]:
sim_df['max sfw'] = max_sfw

In [ ]:
sim_df

In [ ]:
max(sim_df['max scores'])

In [ ]:
sim_df['max scores'].idxmax()

In [ ]:
sim_df.loc[29]

# Clustering and topic modeling skills 

In [ ]:
SFW_16['prof + knowlg + ability'] = ""
for i in range(0, len(SFW_16)):
    SFW_16.loc[i, 'prof + knowlg + ability'] = str(SFW_16.loc[i, 'Proficiency']) + str(SFW_16.loc[i, 'Knowledge']) + str(SFW_16.loc[i, 'Abilities'])

In [ ]:
#SFW
for i in range(0, len(SFW_16)):
    text = SFW_16['prof + knowlg + ability'][i]
    text2 = re.sub('[^A-Za-z0-9]+', ' ', text)
    text3 = text2.strip() 
    SFW_16['prof + knowlg + ability'][i] = text3

In [ ]:
SFW_design = SFW_16[SFW_16['Sector'].str.contains('Design')]

In [ ]:
SFW_design

In [ ]:
len(SFW_design['Skill Title'].unique())

In [ ]:
CF_clean_design = CF_clean[CF_clean['Industry'].str.contains('Design')]

In [ ]:
CF_clean_design

In [ ]:
CF_clean_design.reset_index(inplace=True)

In [ ]:
CF_clean_design['clean_jd'] = ''
for i in range(0, len(CF_clean_design)):
    CF_clean_design['clean_jd'][i] = str(CF_clean_design["Job Description"][i])

In [ ]:
CF_clean_design['clean_jd'][0]

In [ ]:
CF_clean_design['clean_jd'] = CF_clean_design['clean_jd'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [ ]:
CF_clean_design['clean_jd'][0]

In [ ]:
#lowercase all characters
SFW_design['clean_documents'] = SFW_design["prof + knowlg + ability"].fillna('').apply(lambda x: x.lower())

In [ ]:
#lowercase all characters
CF_clean_design['clean_jd'] = CF_clean_design["clean_jd"].fillna('').apply(lambda x: x.lower())

In [ ]:
# tokenization
tokenized_doc = SFW_design['clean_documents'].fillna('').apply(lambda x: x.split())

In [ ]:
stemmer = PorterStemmer()

In [ ]:
# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [stemmer.stem(item) for item in x if item not in stopwords.words('english')])

In [ ]:
tokenized_doc

In [ ]:
#CF
# tokenization
tokenized_doc_cf = CF_clean_design['clean_jd'].fillna('').apply(lambda x: x.split())
# remove stop-words
tokenized_doc_cf = tokenized_doc_cf.apply(lambda x: [stemmer.stem(item) for item in x if item not in stopwords.words('english')])

In [ ]:
tokenized_doc.append(tokenized_doc_cf)

In [ ]:
model = Word2Vec(tokenized_doc)

In [ ]:
words = list(model.wv.vocab)

In [ ]:
words

In [ ]:
# fit a 2d PCA model to the vectors
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)

In [ ]:
pyplot.scatter(result[:, 0], result[:, 1])

In [ ]:
# create a scatter plot of the projection
pyplot.figure(figsize=(30,15))
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
#pyplot.figure(figsize=(30,20))
pyplot.show()

In [ ]:
from wordcloud import WordCloud

In [ ]:
all_words = ' '.join([word for word in words])

In [ ]:
all_words 

In [ ]:
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)
plt.figure(figsize=(15, 8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title("Some frequent words used in the headlines", weight='bold', fontsize=14)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
V = vectorizer.fit_transform(SFW_design['clean_documents'])

In [ ]:
SFW_design['clean_documents']

In [ ]:
#FOR TOPIC MODELING
from sklearn.decomposition import TruncatedSVD
# SVD represent documents and terms in vectors (converting sparse vectors into dense vectors - LSA part)
svd_model = TruncatedSVD(n_components=48, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(V)

In [ ]:
len(svd_model.components_)

In [ ]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
    print(" ")

# Similarity using Spacy

In [ ]:
stemmer = PorterStemmer()

In [ ]:
#SFW
# tokenization
tokenized_doc = SFW_design['clean_documents'].fillna('').apply(lambda x: x.split())
# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [stemmer.stem(item) for item in x if item not in stopwords.words('english')])

In [ ]:
#CF
# tokenization
tokenized_doc_cf = CF_clean_design['clean_jd'].fillna('').apply(lambda x: x.split())
# remove stop-words
tokenized_doc_cf = tokenized_doc_cf.apply(lambda x: [stemmer.stem(item) for item in x if item not in stopwords.words('english')])

In [ ]:
#SFW
# de-tokenization
detokenized_doc = []
for i in range(len(SFW_design)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

In [ ]:
#CF
# de-tokenization
detokenized_doc_cf = []
for i in range(len(CF_clean_design)):
    t = ' '.join(tokenized_doc_cf[i])
    detokenized_doc_cf.append(t)

In [ ]:
SFW_design['stemmed_clean_documents'] = detokenized_doc

In [ ]:
CF_clean_design['stemmed_clean_documents'] = detokenized_doc_cf

In [ ]:
SFW_design

In [ ]:
CF_clean_design

In [ ]:
design_sim = {'cf job titles': [], 'sfw job role': [], 'cf skills': [], 'sfw skill': [], 'sim score': []}

for i in range(0, 10): #for i in range(0, len(CF_clean_design)): 
    scores = []
    sfw_jobs = []
    sfw_skills = []
    for j in range(0, len(SFW_design)):
        sfw = nlp(SFW_design['stemmed_clean_documents'][i])
        cf = nlp(CF_clean_design['stemmed_clean_documents'][j])
        #append items
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(SFW_design['Job Role'][j])
        sfw_skills.append(SFW_design['Skill Title'][j])
    #append to dict    
    design_sim['sim score'].append(scores)
    design_sim['cf job titles'].append(CF_clean_design['Job Title'][i])
    design_sim['cf skills'].append(CF_clean_design['Skills'][i])
    design_sim['sfw job role'].append(sfw_jobs)
    design_sim['sfw skill'].append(sfw_skills)
    print("done" + str(i))

In [ ]:
design_sim

In [ ]:
#3:11 - 3:15 (4mins) for 10 rows
#for 1441 rows will take 9.6hrs

In [ ]:
design_sim_df = pd.DataFrame(design_sim)

In [ ]:
design_sim_df

In [ ]:
max_score_0_id = design_sim_df['sim score'][0].index(max(design_sim_df['sim score'][0]))

In [ ]:
max_score_0_id

In [ ]:
design_sim_df['sfw skill'][0][73]

In [ ]:
design_sim_df['cf skills'][0]

In [ ]:
#gets top 10 sim scores for row 0 in asc order 
top10 = sorted(range(len(design_sim_df['sim score'][0])), key=lambda i: design_sim_df['sim score'][0][i])[-10:]

In [ ]:
#top 10 sfw skills that match cf skills set for row 0
sfwskills = []
for i in top10:
    sfwskills.append(design_sim_df['sfw skill'][0][i])

In [ ]:
sfwskills

In [ ]:
design_sim_df['cf skills'][0]

In [ ]:
#for each sfw find top n cf skills
design_sim = {'sfw job role': [], 'cf job titles': [], 'sfw skill': [], 'cf skills': [], 'sim score': []}

for i in range(0, len(SFW_design)):
    scores = []
    cf_jobs = []
    cf_skills = []
    for j in range(0, len(CF_clean_design)):
        sfw = nlp(SFW_design['stemmed_clean_documents'][i])
        cf = nlp(CF_clean_design['stemmed_clean_documents'][j])
        #append items
        scores.append(sfw.similarity(cf))
        cf_jobs.append(CF_clean_design['Job Title'][j])
        cf_skills.append(CF_clean_design['Skills'][j])
    #append to dict    
    design_sim['sim score'].append(scores)
    design_sim['cf job titles'].append(cf_jobs)
    design_sim['cf skills'].append(cf_jobs)
    design_sim['sfw job role'].append(SFW_design['Job Role'][i])
    design_sim['sfw skill'].append(SFW_design['Skill Title'][i])
    

# Similarity using LSA

In [ ]:
SFW_16

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(SFW_16['clean_documents'])

In [ ]:
#FOR SIMILARITY
from sklearn.decomposition import TruncatedSVD
# SVD represent documents and terms in vectors (converting sparse vectors into dense vectors - LSA part)
svd_model2 = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa2 = svd_model2.fit_transform(X)

In [ ]:
JD = CF_clean['Job Description'][0]

In [ ]:
text2 = re.sub('[^A-Za-z0-9]+', ' ', JD)

In [ ]:
text2

In [ ]:
text3 = text2.lower().split()

In [ ]:
text4 = [word for word in text3 if not word in stopwords.words()]

In [ ]:
text4 = [stemmer.stem(word) for word in text4]

In [ ]:
# de-tokenization
t = ' '.join(text4)

In [ ]:
t = [t]

In [ ]:
t

In [ ]:
jd = vectorizer.fit_transform(t)

In [ ]:
svd_model2 = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa_jd2 = svd_model2.fit_transform(jd)

In [ ]:
cosine_values = []
similarity_perc = []
for i in range(0, len(lsa2)):
    cos = scipy.spatial.distance.cosine(lsa2[i], lsa_jd2)
    cosine_values.append(cos)
    similarity_perc.append(round((1-cos)*100,2))

In [ ]:
similarity_perc

In [ ]:
m = max(similarity_perc)
m

In [ ]:
m = max(similarity_perc)
index_of_maximum = np.where(similarity_perc == m)

In [ ]:
index_of_maximum

In [ ]:
SFW_16.loc[920]

In [ ]:
CF_clean.loc[0]

In [ ]:
cf_skills = []
for i in range(0, len(df_cf['Skills'])):
    cf_skills.append(literal_eval(df_cf['Skills'][i]))

In [ ]:
cf_skills

In [ ]:
cf_skills_lower = [[w.lower() for w in line] for line in cf_skills]

In [ ]:
cf_skills_lower

In [ ]:
len(cf_skills_lower)

In [ ]:
#how to get unique skills from list of lists
cf_skills_unique = []
for x in cf_skills_lower:
    if x not in cf_skills_unique:
        cf_skills_unique.append(x)

In [ ]:
len(output)

In [ ]:
print(cf_skills_unique)